Use this for playcounts: https://github.com/entriphy/sp-playcount-librespot

In [ ]:
import json
import datetime
import requests

In [ ]:
spotify_client_id = ''
spotify_client_secret = ''

token = requests.post(
    'https://accounts.spotify.com/api/token',
    data={'grant_type': 'client_credentials'},
    auth=requests.auth.HTTPBasicAuth(
        spotify_client_id,
        spotify_client_secret
    )
).json()

In [ ]:
r = requests.get(
    'https://api.spotify.com/v1/tracks/11dFghVXANMlKmJXsNCbNl',
    headers={'Authorization': f'Bearer {token["access_token"]}'}
)

r.json()

In [ ]:
r = requests.get('https://api.t4ils.dev/albumPlayCount?albumid=6Lq1lrCfkpxKa4jCo5gKWr')
r.json()

In [ ]:
l = []
d = datetime.date(1970, 1, 1)
for i in range(10):
    x = {
        'date': str(d),
        'track1Id': '',
        'track1Name': '',
        'track1Listens': '',
        'track2Id': '',
        'track2Name': '',
        'track2Listens': '',
        'finalized': ''
    }
    l.append(x)
    d += datetime.timedelta(days=1)
    

In [ ]:
json.dumps(l)